In [1]:
# https://androidkt.com/use-saved-pytorch-model-to-predict-single-and-multiple-images/
# https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-predict-new-samples-with-your-pytorch-model.mdS
# https://towardsdatascience.com/optimize-pytorch-performance-for-speed-and-memory-efficiency-2022-84f453916ea6
# https://www.kaggle.com/code/andradaolteanu/pytorch-rnns-and-lstms-explained-acc-0-99#3.-RNN-with-1-Layer-%F0%9F%93%98
# https://www.kaggle.com/code/omershect/learning-pytorch-lstm-deep-learning-with-m5-data/comments
# https://curiousily.com/posts/time-series-anomaly-detection-using-lstm-autoencoder-with-pytorch-in-python/
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import time
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Available Device is: ', device)

Available Device is:  cpu


In [36]:
df = pd.read_csv('./robotdatacollection3/ep2.csv')
feature_list = ['Fy']
TIMESTEP = 50

In [37]:
df_new = df[feature_list].to_numpy()
print('Shape before adding dimension:',df_new.shape)
df_new = np.expand_dims(df_new, axis=1)
print('Shape AFTER adding dimension:',df_new.shape)

Shape before adding dimension: (5351, 1)
Shape AFTER adding dimension: (5351, 1, 1)


### Try using Or's approach

In [38]:
# shape the input to 
# batch=1, timestep=values, features=1

In [39]:
# we set the data in a window form, in case of timestep = 5
# i=0: X_0=[x(0)..x(5)], y=x(6)

def to_sequence(data, timesteps=1):
    n_features=data.shape[2]
    x = []
    y = []
    for i in range(len(data)-timesteps):
        # takes a window of data of specified timesteps
        
        _x = data[i:(i+timesteps)]
        _x = _x.reshape(timesteps, n_features)
#         print(_x.shape)
        _y = data[i+timesteps]
        _y = _y.reshape(n_features)
#         print(_y.shape)
        x.append(_x)
        y.append(_y)

        
    return np.array(x), np.array(y)

In [40]:
x_train, y_train = to_sequence(df_new, timesteps=TIMESTEP)

In [41]:
# print(x_train.shape)
# print(y_train.shape)

In [42]:
# x_train = df_new #np.ones((1,5351))#
# y_train = np.ones((5351,1))# np.ones((5351,1))# 

In [43]:
trainX = torch.Tensor(x_train)
trainy = torch.Tensor(y_train)
print(trainX.shape)
print(trainy.shape)

torch.Size([5301, 50, 1])
torch.Size([5301, 1])


**Pytorch uses Tensors as inputs to the model, so we convert numpy to Tensor**

### LSTM


LSTM layers work on 3D data with the following structure `(sequence/batch, timestep, feature)`.

* **input_dim**: Number of samples in the data. In our case we do not use batches but instead one sample at a time. Each sample has TIMESTEP size of data in it. 
* **hidden_dim**: Number of LSTM cells
* **layer_dim**: Number of LSTM hidden layers
* **output_dim**: Number of outputs = feature number (in case of regression task)




https://cnvrg.io/pytorch-lstm/

In [44]:
#Bidirectional LSTM: extensions that can improve performance. They train model forward and backward on the same input
# much slower, good for NLP 

class LSTMModel(nn.Module):
    def __init__(self, batch_size, sequence_length, input_size, hidden_dim, output_dim):
        super(LSTMModel, self).__init__() # inherit from nn.Module
        
        # N: sample size, each sample containts sequence of length L
        self.batch_size = batch_size
        # L: sequence length 
        self.sequence_length = sequence_length
        # Hin: input size, number of features in the input X
        self.input_size = input_size
        
        # hidden dims
        self.hidden_dim = hidden_dim
        
        # building LSTM
        # setting batch_first=True: input and output order:(N,L,Hin)
        self.lstm = nn.LSTM(input_size, hidden_dim, batch_first=True)
        #         self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        # output layer (OK) 
        self.fc = nn.Linear(hidden_dim, output_dim)

        
    def forward(self, x):
        # initialize hidden state with zeros
        hidden_state = torch.zeros(1, self.batch_size, self.hidden_dim)
        # initialize cell state
        cell_state = torch.zeros(1, self.batch_size, self.hidden_dim)
        # input order:(N,L,Hin) for batch_first = True  
        # view() reshapes tensor without copying memory
        x = x.view(self.batch_size, self.sequence_length, self.input_size)
#         print(x.shape)

        # output, (h_n, c_n): 
        # we need to detach tensor from current graph
        # if we do not, we will backpop all the way to the start even after going through another batch
        out, (last_hidden_state, last_cell_state) = self.lstm(x, (hidden_state, cell_state))

#         out, (hn, cn) = self.lstm(X, (h_0, c_0))
#         print('output shape', out.shape)
        
        # index hidden state of last time step
        # out.size() -> 100,28,100
        # out[:,-1,:] -> 100, 100-> just want last time step hidden states
        out = self.fc(out[:,-1,:])
#         print('reshape',out.shape)
        # out.size() --> 100,10
        return out

* `batch_size`: N-different measurements, each contains sequence of length L
* `sequence_length`: L: length of each time series sequence
* `input_size`: Hin: number of features
* `hidden_dim`: number of units in LSTM layer
* `output_dim`: output size/ how many predictions at a time

In [32]:
trainX.shape

torch.Size([5301, 50, 2])

In [48]:
bs = trainX.shape[0]
sl = trainX.shape[1]

In [49]:
lstm_model = LSTMModel(batch_size=bs,
                       sequence_length=sl, 
                       input_size=1, 
                       hidden_dim=15, 
                       output_dim=1)






# input_dim = 1#  timestep
# hidden_dim = 5 # LSTM layer 5 units
# layer_dim = 1 # one LSTM layer
# output_dim = 1 # we outout one prediction at a time

# lstm_model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)

In [50]:
lstm_model

LSTMModel(
  (lstm): LSTM(1, 15, batch_first=True)
  (fc): Linear(in_features=15, out_features=1, bias=True)
)

In [51]:
# regression, calculates the loss function
criterion = torch.nn.MSELoss()
# updates the weights and biases to reduce loss
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)


# Training LSTM for each epoch we
for epoch in range(50):
    
    # enable train mode
    lstm_model.train(True)
    
    # clear gradients with respect to params, always before backprop
    optimizer.zero_grad(set_to_none=True)
    
    # track history if only in train
    with torch.set_grad_enabled(True):
        
        # forward pass to get outputs/ make predictions for that batch
        output = lstm_model(trainX)

        # calculate loss and its gradients
        loss = criterion(output, trainy)
        # getting gradients w.r.t. params
        loss.backward()

        #updating params/ weights
        optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss {loss}")
    

Epoch 0, Loss 0.744158148765564
Epoch 10, Loss 0.6845821142196655
Epoch 20, Loss 0.6352720260620117
Epoch 30, Loss 0.5895519852638245
Epoch 40, Loss 0.5426255464553833


In [52]:
output.shape

torch.Size([5301, 1])

In [53]:
trainy.shape

torch.Size([5301, 1])

In [54]:
t = 0

times_dnn = []
lstm_model.eval()
torch.no_grad()

while t < 10:
    t_start = time.time()
    
    with torch.no_grad():
        y_pred = lstm_model(trainX)
        
    dt = time.time()-t_start
    print(dt)
    
    times_dnn.append(dt)
    t += 1

0.09427380561828613
0.07773137092590332
0.07767796516418457
0.07771062850952148
0.07795238494873047
0.07773184776306152
0.0729987621307373
0.05622577667236328
0.05527138710021973
0.05552220344543457


In [55]:
trainX.shape

torch.Size([5301, 50, 1])

In [65]:
test = torch.randn(1,2)
print(test.shape)

torch.Size([1, 2])


In [ ]:
lstm_model = LSTMModel(batch_size=bs,
                       sequence_length=sl, 
                       input_size=1, 
                       hidden_dim=15, 
                       output_dim=1)

In [64]:
lstm_model(test)

RuntimeError: shape '[5301, 50, 1]' is invalid for input of size 2